# 

In [1]:
from captum.attr import LayerIntegratedGradients

In [2]:
from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig

In [3]:
# there are some warning from transformer
# due to its verbose, disable

from transformers import logging
logging.set_verbosity(40)

In [4]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [6]:
import pandas as pd
from tqdm.autonotebook import tqdm

In [7]:
from linevul_model import Model
from linevul_helpers import TextDataset
from linevul_extra import extract_line_attention

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [9]:
config = RobertaConfig.from_pretrained('microsoft/codebert-base')
config.num_labels = 1
config.num_attention_heads = 12

In [10]:
# get from LineVul
checkpoint = '/home/hqn650/LineVul/linevul/saved_models/checkpoint-best-f1/12heads_linevul_model.bin'

In [11]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

In [12]:
pre_train = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', 
                                                             config=config, 
                                                             ignore_mismatched_sizes=True)

In [13]:
from dataclasses import dataclass

@dataclass
class Args:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_gpu = torch.cuda.device_count()
    use_non_pretrained_model = False
    block_size = 512
    test_data_file = '/home/hqn650/LineVul/data/big-vul_dataset/test.csv'
    code_length=256
    do_local_explanation=True
    reasoning_method='attention'
    seed=42
    num_attention_heads=12
    do_sorting_by_line_scores=False
    do_sorting_by_pred_prob=False
    top_k_constant=10
    use_word_level_tokenizer=False
    eval_batch_size=512
    
args = Args()

In [14]:
model = Model(pre_train, config, tokenizer, args)

In [15]:
model.load_state_dict(torch.load(checkpoint, map_location=args.device))
model.to(args.device)

Model(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [16]:
test_dataset = TextDataset(tokenizer, args, file_type='test')

  0%|          | 0/18864 [00:00<?, ?it/s]

In [17]:
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args.eval_batch_size, num_workers=0)

In [18]:
# multi-gpu evaluate
if args.n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [21]:
def linevul_predict(model, dataloader, threshold=0.5):
    model.eval()
    logits=[]  
    y_trues=[]
    for batch in test_dataloader:
        (inputs_ids, labels) = [x.to(args.device) for x in batch]
        with torch.no_grad():
            lm_loss, logit = model(input_ids=inputs_ids, labels=labels)
            logits.append(logit.cpu().numpy())
            y_trues.append(labels.cpu().numpy())
    # calculate scores
    logits = np.concatenate(logits, 0)
    y_trues = np.concatenate(y_trues, 0)
    y_preds = logits[:, 1] > threshold
    acc = accuracy_score(y_trues, y_preds)
    recall = recall_score(y_trues, y_preds)
    precision = precision_score(y_trues, y_preds)   
    f1 = f1_score(y_trues, y_preds)             
    result = {
        "test_accuracy": float(acc),
        "test_recall": float(recall),
        "test_precision": float(precision),
        "test_f1": float(f1),
        "test_threshold":threshold,
    }
    return result, y_trues, y_preds

In [22]:
result, y_trues, y_preds = linevul_predict(model, test_dataloader)

/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [23]:
result

{'test_accuracy': 0.9909351145038168,
 'test_recall': 0.8635071090047394,
 'test_precision': 0.9712153518123667,
 'test_f1': 0.9141996989463121,
 'test_threshold': 0.5}

In [24]:
correct_indices = np.where((y_trues == y_preds))
correct_indices = list(correct_indices[0])

In [25]:
tp_indices = np.where((y_trues == y_preds) & (y_trues == 1))
tp_indices = list(tp_indices[0])

In [26]:
# after identify true positive sample, create new loader for explaination

dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

In [27]:
df = pd.read_csv(args.test_data_file)

In [28]:
top_k_constant = [args.top_k_constant]

In [29]:
model.eval()
index = 0
progress_bar = tqdm(dataloader, total=len(dataloader))
extract_list = []
for mini_batch in progress_bar:
    if index in tp_indices:
        (input_ids, labels) = mini_batch
        ids = input_ids[0].detach().tolist()
        all_tokens = tokenizer.convert_ids_to_tokens(ids)
        all_tokens = [token.replace("Ġ", "") for token in all_tokens]
        all_tokens = [token.replace("ĉ", "Ċ") for token in all_tokens]
        with torch.no_grad():
            prob, attentions = model(input_ids=input_ids, output_attentions=True)
        lines_with_score, n_lines = extract_line_attention(attentions, all_tokens)
        extract_list.append((lines_with_score, n_lines))
    index += 1

  0%|          | 0/18864 [00:00<?, ?it/s]

In [30]:
len(extract_list)

911

In [ ]:
lines_with_score

In [ ]:
sorted_lines = sorted(lines_with_score, key=lambda x: x[2], reverse=True)
sorted_lines[:int(0.15*line_idx)]

In [ ]:
import codecs

context = df.iloc[99]['func_before']
modified_context = codecs.decode(context, 'unicode_escape')

new_variable = modified_context.replace(r'\n', '\n')

print(new_variable)


In [ ]:
len(lines_with_score)

In [ ]:
df.iloc[99]